In [5]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from fake_useragent import UserAgent
from dotenv import load_dotenv
from tqdm import tqdm
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from Levenshtein import distance
load_dotenv()

True

In [ ]:
dataset = pd.read_csv("./recepies.csv", delimiter='\t', index_col="id")

In [7]:
dataset.head()

,name,composition,cooking_type,Инструкции,dish_type,Дата,photo,source,composition_inter
id,,,,,,,,,
0,рассольник классический с перловкой и солеными...,"[{'Перловка': 0.1, 'unit': 'стак. (200 мл)'}, ...","варка,жарка",Подготовить указанные ингредиенты для приготов...,первое,05.06.2015,photo_1000menu_1.jpg,https://1000.menu/cooking/33395-rassolnik-s-pe...,"[{'product_id': 4253, 'name_source': 'Перловая..."
1,Суп пюре из белокочаной капусты,"[{'Капуста белокочанная': 50.0, 'unit': 'гр'},...",варка,"Необходимые ингредиенты\r\nНарезаем лук, морко...",первое,27.06.2015,photo_1000menu_2.jpg,https://1000.menu/cooking/25399-sup-pure-iz-be...,"[{'product_id': 2286, 'name_source': 'Капуста ..."
2,Постные щи из квашеной капусты,"[{'Капуста квашеная': 116.7, 'unit': 'гр'}, {'...","варка,жарка,тушение","Честно признаюсь, у меня не было репы на момен...",первое,12.02.2013,photo_1000menu_3.jpg,https://1000.menu/cooking/5159-postnje-shchi,"[{'product_id': 0, 'name_source': 'Капуста ква..."
3,Тюря- простой суп быстро и вкусно,"[{'Квас': 0.2, 'unit': 'л'}, {'Лук репчатый': ...",сырое,"\r\nНачинаем мы приготовление тюри с того, что...",первое,02.03.2011,photo_1000menu_4.jpg,https://1000.menu/cooking/5085-turya,"[{'product_id': 0, 'name_source': 'Квас', 'uni..."
4,Фасолевый суп из красной фасоли,"[{'Вода': 0.3, 'unit': 'л'}, {'Картошка': 0.3,...",варка,Подготовить ингредиенты. Для приготовления суп...,первое,28.01.2013,photo_1000menu_5.jpg,https://1000.menu/cooking/38765-fasolevyi-sup-...,"[{'product_id': 828, 'name_source': 'Вода', 'u..."


In [8]:
nan_counts = dataset.isna().sum()
print(nan_counts)

name                 0
composition          0
cooking_type         0
Инструкции           0
dish_type            0
Дата                 0
photo                0
source               0
composition_inter    0
dtype: int64


Great! No missing values

In [9]:
dataset.describe()

,name,composition,cooking_type,Инструкции,dish_type,Дата,photo,source,composition_inter
count,27884,27884,27884,27884,27884,27884,27884,27884,27884
unique,27792,27825,16,27827,5,3655,27884,27850,27824
top,Оссобуко,"[{'Яйцо куриное': 0.5, 'unit': 'штуки'}]",сырое,"1. Творог протереть через сито, можно пробить ...",перекус,13.02.2017,photo_eda_28468.jpg,https://povar.ru/recipes/novogodnii_salat_elka...,"[{'product_id': 6652, 'name_source': 'Шампиньо..."
freq,3,3,12701,3,13224,20,1,3,3


Let's add "is_vegan" column and drop some unneccessary data

In [10]:
processed_dataset = dataset.drop(columns=["Дата", "composition_inter"])

In [11]:
processed_dataset["composition"][26470]

"[{'Мангольд': 'по вкусу', 'unit': 'по вкусу'}, {'Яблоки «гренни-смит»': 0.2, 'unit': 'штука'}, {'Огурцы': 0.2, 'unit': 'штука'}, {'Оливки': 0.1, 'unit': 'стакана'}, {'Зеленый лук': 0.1, 'unit': 'пучка'}, {'Авокадо': 0.1, 'unit': 'штуки'}, {'Капуста брокколи': 0.2, 'unit': 'стакан'}, {'Фисташки': 12.5, 'unit': 'г'}, {'Сыр фета': 12.5, 'unit': 'г'}, {'Оливковое масло': 0.1, 'unit': 'стакана'}, {'Лимонный сок': 0.4, 'unit': 'столовые'}, {'Дижонская горчица': 0.1, 'unit': 'чайной'}, {'Мед': 0.1, 'unit': 'чайной'}, {'Базилик': 0.1, 'unit': 'пучка'}, {'Чеснок': 0.2, 'unit': 'зубчик'}, {'Соль': 'по вкусу', 'unit': 'по вкусу'}, {'Перец черный молотый': 'по вкусу', 'unit': 'по вкусу'}]"

As we can see, composition is just a string, let's change that

In [12]:
processed_dataset["composition"] = processed_dataset["composition"].apply(lambda x: eval(x))
processed_dataset["composition"][26470]

[{'Мангольд': 'по вкусу', 'unit': 'по вкусу'},
 {'Яблоки «гренни-смит»': 0.2, 'unit': 'штука'},
 {'Огурцы': 0.2, 'unit': 'штука'},
 {'Оливки': 0.1, 'unit': 'стакана'},
 {'Зеленый лук': 0.1, 'unit': 'пучка'},
 {'Авокадо': 0.1, 'unit': 'штуки'},
 {'Капуста брокколи': 0.2, 'unit': 'стакан'},
 {'Фисташки': 12.5, 'unit': 'г'},
 {'Сыр фета': 12.5, 'unit': 'г'},
 {'Оливковое масло': 0.1, 'unit': 'стакана'},
 {'Лимонный сок': 0.4, 'unit': 'столовые'},
 {'Дижонская горчица': 0.1, 'unit': 'чайной'},
 {'Мед': 0.1, 'unit': 'чайной'},
 {'Базилик': 0.1, 'unit': 'пучка'},
 {'Чеснок': 0.2, 'unit': 'зубчик'},
 {'Соль': 'по вкусу', 'unit': 'по вкусу'},
 {'Перец черный молотый': 'по вкусу', 'unit': 'по вкусу'}]

In [25]:
import string

ingredients = set()
for recipe in processed_dataset["composition"]:
    for x in recipe:
        ings = [y.lower().strip().replace('ё', 'е') for y in x.keys() if y != "unit"]
        translator = str.maketrans('', '', string.punctuation.replace('-', ''))
        ings = [ing.translate(translator) for ing in ings]
        for ing in ings:
            if ',' in ing:
                ings.remove(ing)
                ings.extend(ing.split(','))
        ingredients.update([i.strip() for i in ings])
len(ingredients)

4704

Okay, that's quite a lot of products, let's search for duplicates

In [26]:
import itertools

# Function to find the closest matches using Levenshtein distance
def find_closest_matches(ingredients, threshold=2):
    duplicates = []
    for ing1, ing2 in itertools.combinations(ingredients, 2):
        if distance(ing1, ing2) <= threshold:
            duplicates.append((ing1, ing2))
    return duplicates

# Find duplicates with a threshold of 2
duplicates = find_closest_matches(ingredients, threshold=2)
len(duplicates)

642

In [27]:
duplicates

[('свиные кости', 'утиные кости'),
 ('яблоко', 'молоко'),
 ('яблоко', 'яблоки'),
 ('кефир 1-ый', 'кефир 32-ый'),
 ('кефир 1-ый', 'кефир 25-ый'),
 ('улитки', 'сливки'),
 ('улитки', 'оливки'),
 ('мандарин', 'маргарин'),
 ('мандарин', 'мандарины'),
 ('молоко 6-ное', 'молоко 35-ное'),
 ('молоко 6-ное', 'молоко 2-ое'),
 ('молоко 6-ное', 'молоко 32-ное'),
 ('молоко 6-ное', 'молоко 1-ое'),
 ('сушеные грибы', 'соленые грибы'),
 ('перец', 'перно'),
 ('перец', 'херес'),
 ('миноги', 'иноки'),
 ('сок', 'сыр'),
 ('сок', 'хек'),
 ('сок', 'лук'),
 ('сок', 'сода'),
 ('сок', 'ром'),
 ('сок', 'соль'),
 ('сок', 'сом'),
 ('сок', 'саке'),
 ('сок', 'мак'),
 ('лярд', 'лед'),
 ('говяжьи щечки', 'говяжьи почки'),
 ('сливки 38-ные', 'сливки 35-ные'),
 ('сливки 38-ные', 'сливки 11-ные'),
 ('сливки 38-ные', 'сливки 30-ные'),
 ('сливки 38-ные', 'сливки 25-ные'),
 ('сливки 38-ные', 'сливки 20-ные'),
 ('сливки 38-ные', 'сливки 40-ные'),
 ('сливки 38-ные', 'сливки 10-ные'),
 ('сливки 38-ные', 'сливки 9-ные'),
 ('слив

Okay, after looking deeper into data there are not much of a true duplicates except for singular and plural forms. Let's skip that part.

In [28]:
ingredients_dataframe = pd.DataFrame(ingredients, columns=["ingredient"])
ingredients_dataframe.to_csv("ingredients.csv", index=False)
ingredients_dataframe.head()

,ingredient
0,сыр творожный
1,кабачковая икра
2,цицак
3,сушеная зелень
4,клементины


To speed up the process we will utilize LLM to decide if the product vegan/vegetarian/non-vegan

In [30]:
llm = ChatOpenAI(model="gpt-4o-mini")

class VeganTest(BaseModel):
    result: int = Field(description="Ты получишь на вход название ингредиента, на выходе нужно вернуть 1, если ингредиент веганский, 0, если нет, -1 если он подходит вегетарианцам, но не подходит веганам. Если ты не знаешь ответа, верни 0.")

structured_llm = llm.with_structured_output(VeganTest)

async def is_vegan(ingredient: str) -> int:
    try:
        res = await structured_llm.ainvoke(ingredient)
        return res
    except Exception as e:
        print(e)
        return 0

In [31]:
ingredients_vegan = []
for ingredient in tqdm(ingredients_dataframe["ingredient"]):
    ingredients_vegan.append(await is_vegan(ingredient))

ingredients_dataframe["vegan"] = ingredients_vegan
ingredients_dataframe.head()

  5%|▌         | 243/4704 [06:00<1:55:00,  1.55s/it]

Error code: 403 - {'error': {'message': 'openai/gpt-4o-mini requires moderation on Azure. Your input was flagged for "sexual". No credits were charged.', 'code': 403, 'metadata': {'reasons': ['sexual'], 'flagged_input': 'цыплячьи грудки', 'provider_name': 'Azure', 'model_slug': 'openai/gpt-4o-mini'}}}


100%|██████████| 4704/4704 [2:00:36<00:00,  1.54s/it]  


,ingredient,vegan
0,сыр творожный,result=-1
1,кабачковая икра,result=1
2,цицак,result=0
3,сушеная зелень,result=1
4,клементины,result=1


In [32]:
ingredients_dataframe.to_csv("ingredients.csv", index=False)

In [35]:
ingredients_dataframe['vegan'].describe()

count         4704
unique           4
top       result=1
freq          2778
Name: vegan, dtype: object